In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from numpy.linalg import inv
from aux_fun import *

In [2]:
# Display all cell results (not only last one)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
dados = pd.read_csv('https://raw.githubusercontent.com/Cayan-Portela/ceub/main/dados/df_notas.csv')
dados.head()

,nota,hrs_estudo,faltas
0,9.0,4.0,2
1,7.0,4.0,1
2,5.5,2.0,4
3,8.0,3.0,1
4,2.0,0.5,5


In [4]:
modelo_1 = smf.ols("nota ~ hrs_estudo", data=dados).fit()
modelo_2 = smf.ols("nota ~ hrs_estudo + faltas", data=dados).fit()

print(f"Intercepto: {modelo_1.params['Intercept']:.2f} | Horas de estudo: {modelo_1.params['hrs_estudo']:.2f}")
print(f"Intercepto: {modelo_2.params['Intercept']:.2f} | Horas de estudo: {modelo_2.params['hrs_estudo']:.2f} | Faltas: {modelo_2.params['faltas']:.2f}")

Intercepto: 2.30 | Horas de estudo: 1.50
Intercepto: 3.50 | Horas de estudo: 1.27 | Faltas: -0.26


#### Relembrando

$Y = X\hat{\beta} + \epsilon$

$\hat{\beta} = (X'X)^{-1}X'Y$

$\hat{Y} = X\hat{\beta}$

In [5]:
# Estimando betas
Y = dados['nota']
X_mat = matriz_x(colunas = ['hrs_estudo', 'faltas'], dados=dados)
beta = inv(X_mat.T @ X_mat) @ X_mat.T @ Y

Regressão Linear Múltipla:

$Nota = \beta_{0a} + \beta_{1a}\text{Estudo} + \beta_{2a}\text{Faltas} + \epsilon_a$

em que $\beta_{1a} = 1.26$

Podemos chegar no paramaetro $\beta_{estudo} = 1.26$ utilizando o residuo de uma regressao de _Estudo_ em _Faltas_.

Modelo 1)
  - $Estudo = \beta_{0b} + \beta_{1b}\text{Faltas} + \epsilon_b$
  
Modelo 2)
  - $Nota = \beta_{0c} + \beta_{1c}\epsilon_b + \epsilon_c$

Assim, $\beta_{1c} = \beta_{estudo}$

In [6]:
def reg_linear(x, y, dados):
    y_ = dados[y]
    x_mat = matriz_x(colunas=x, dados=dados)
    beta = inv(x_mat.T @ x_mat) @ x_mat.T @ y_
    y_hat = x_mat @ beta
    residuo = y_ - y_hat

    return beta, y_hat, residuo

In [7]:
# Modelo 1)
beta_1, y_hat_1, residuo_1 = reg_linear(x=['faltas'], y=['hrs_estudo'], dados=dados)

In [8]:
# Modelo 2)
beta_2, y_hat_2, residuo_2 = reg_linear(x=['residuo'], y=['nota'], dados=dados.assign(residuo = residuo_1))

In [9]:
# Comparando parametros
print(f"Horas de estudo {modelo_2.params['hrs_estudo']:.4f} (reg. linear multipla)")
print(f"Horas de estudo {beta_2['nota'][1]:.4f} (regressao com residuos)")

Horas de estudo 1.2660 (reg. linear multipla)
Horas de estudo 1.2660 (regressao com residuos)


Mesmo resultado utilizando `statsmodels`

In [10]:
modelo_1 = smf.ols("hrs_estudo ~ faltas", data=dados).fit()

modelo_2 = smf.ols("nota ~ estudo_residuo", data=dados.assign(estudo_residuo=modelo_1.resid)).fit()
print(f"Horas de estudo: {modelo_2.params['estudo_residuo']:.4f}")

Horas de estudo: 1.2660
